# Test Cookie

In [1]:
import requests as rq
import tensorflow.keras.models
import json
import numpy as np
import cv2
import os

## get asp.net_sessionid

In [2]:
resp=rq.get('http://ecardfw.upc.edu.cn:20086/')

In [3]:
cookies={'ASP.NET_SessionId':resp.cookies['ASP.NET_SessionId']}

In [4]:
import time
validate_code_time=str(time.time()).replace('.','')
validate_code_url='http://ecardfw.upc.edu.cn:20086/Login/GetValidateCode?time='.format(validate_code_time)
validate_code_resp=rq.get(validate_code_url,cookies=cookies)

In [5]:
with open('./validate_code_tmp.jpg','wb') as f:
    f.write(validate_code_resp.content)
    f.flush()
    validate_img=cv2.imread(f.name)
os.remove('./validate_code_tmp.jpg')

### all

In [34]:
resp=rq.get('http://ecardfw.upc.edu.cn:20086/')
cookies={'ASP.NET_SessionId':resp.cookies['ASP.NET_SessionId']}
import time
validate_code_time=str(time.time()).replace('.','')
validate_code_url='http://ecardfw.upc.edu.cn:20086/Login/GetValidateCode?time='.format(validate_code_time)
validate_code_resp=rq.get(validate_code_url,cookies=cookies)
with open('./validate_code_tmp.jpg','wb') as f:
    f.write(validate_code_resp.content)
    f.flush()
    validate_img=cv2.imread(f.name)
os.remove('./validate_code_tmp.jpg')

## calculate the calidate code

In [35]:
headers={
    'Host': 'ecardfw.upc.edu.cn:20086',
    'Proxy-Connection': 'keep-alive',
    'Content-Length': '72',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'http://ecardfw.upc.edu.cn:20086',
    'Referer': 'http://ecardfw.upc.edu.cn:20086/',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9,zh-TW;q=0.8,en-US;q=0.7,en;q=0.6,fi;q=0.5',
}

In [6]:
models=tensorflow.keras.models.load_model('./result.h5')

In [36]:
def decode(y):
    y = np.argmax(np.array(y), axis=2)[:,0]
    return ''.join([str(x) for x in y])
input_width=109
input_height=41
validate_img=cv2.resize(validate_img,(input_width,input_height)).reshape((1,input_height,input_width,3))
pre=models.predict(validate_img/255)
validate_code=decode(pre)
print(validate_code)

18682


In [37]:
post_data={'sno':'1805010118','pwd':'MDUxNTEw','ValiCode':validate_code,'remember':'1','uclass':'1','json':'true'}
login_resp=rq.post('http://ecardfw.upc.edu.cn:20086/Login/LoginBySnoQuery',data=post_data,cookies=cookies,headers=headers)

In [58]:
import base64
s='051510'
base64.b64encode(s.encode('utf-8')).decode('ascii')

'MDUxNTEw'

In [76]:
a=login_resp.content
a=json.loads(a)
a['IsSucceed'] == '1'

False

In [86]:
a=Exception('login failed','captcha_error')
a.args[1]

'captcha_error'